# 文字提取流程

In [1]:
# 基础环境
import os
import sys
import fire
import shutil
import comet_ml
import pandas as pd
from tqdm import tqdm
from icecream import ic
from datetime import datetime
from pathlib import PurePath, Path
from PIL import Image, ImageDraw

# FILE = Path(__file__).resolve()
FILE = Path("/home/aor/projects/Developing/eldsich/pdf_parser").resolve()
ROOT = str(FILE.parents[0])

In [2]:
# 初始化paddle ocr
from paddleocr import PaddleOCR
paddle_ocr_model_path = ROOT + "/weights/paddle_ocr/ch_PP-OCRv4_det_server_infer/"
ocr = PaddleOCR(use_angle_cls=True, lang="ch", det_model_dir=paddle_ocr_model_path)

[2024/05/15 23:12:41] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/aor/projects/Developing/eldsich/weights/paddle_ocr/ch_PP-OCRv4_det_server_infer/', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/aor/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, 

In [3]:
# 识别图片文件后缀
def get_file_extension_in_dir(dir_path: str, file_name: str):
    for file in os.listdir(dir_path):
        name, extension = os.path.splitext(file)
        if name == file_name:
            return extension
    return None

# 读取结构识别结果
def read_structure_data(dir : str):
    struct_data = pd.read_csv(os.path.join(dir, "recognized.csv"), header=0)
    return struct_data

def paddle_ocr(image_path):
    result = ocr.ocr(image_path, cls=True)
    ocr_result = []
    for idx in range(len(result)):
        res = result[idx]
        if res:
            for line in res:
                ocr_result.append(line[1][0])
    return "".join(ocr_result)

def create_cell_img(dir : str):
    # 刷新结果目录
    crop_ocr_dir = os.path.join(dir, "ocr")
    if os.path.exists(crop_ocr_dir):
        shutil.rmtree(crop_ocr_dir)
    os.makedirs(crop_ocr_dir)

    # 数据读取
    df = read_structure_data(dir)
    img_ext = get_file_extension_in_dir(dir, "origin")
    img = Image.open(os.path.join(dir, "origin" + img_ext))

    # 组织结构并在横纵方向排序
    # fixme: 这里是按网格处理，没有真正拿到实际表格结构
    cols = []
    rows = []        
    for idx, box in df.iterrows():
        if box['labels'] == 1:
            cols.append([box["box0"], box["box1"], box["box2"], box["box3"]])
        if box['labels'] == 2:
            rows.append([box["box0"], box["box1"], box["box2"], box["box3"]])
    sorted_cols = sorted(cols, key=lambda x: x[0])
    sorted_rows = sorted(rows, key=lambda x: x[1])

    for row_index, row in enumerate(sorted_rows):
        for col_index, col in enumerate(sorted_cols):
            rect = [int(col[0]), int(row[1]), int(col[2]), int(row[3])]
            cell_image = img.crop(rect)
            cell_image_path = f'{crop_ocr_dir}/cell_{row_index}_{col_index}{img_ext}'
            cell_image.save(cell_image_path)

def ocr_cells(dir : str):
    cell_ocr_dir = os.path.join(dir, "ocr")
    cell_files = os.listdir(cell_ocr_dir)
    if "ocr.xlsx" in cell_files:
        cell_files.remove("ocr.xlsx")

    max_x, max_y = 0, 0
    for file in cell_files:
        cell_name = file.split(".")[0]
        x, y = [int(_) for _ in cell_name.split("_")[1:]]
        max_x = max(x, max_x)
        max_y = max(y, max_y)

    ocr_result = [["" for _ in range(max_y + 1)] for _ in range(max_x + 1)]
    for file in cell_files:
        cell_name = file.split(".")[0]
        x, y = [int(_) for _ in cell_name.split("_")[1:]]
        
        cell_img_dir = os.path.join(cell_ocr_dir, file)
        cell_text = paddle_ocr(cell_img_dir)
        ocr_result[x][y] = cell_text

    df = pd.DataFrame(ocr_result)
    excel_path = os.path.join(cell_ocr_dir, "ocr.xlsx")
    df.to_excel(excel_path, index=False)


In [4]:
def ocr_page(page_dir : str):
    # 重建目标文件夹
    ocr_dir = os.path.join(page_dir, "ocr")
    if os.path.exists(ocr_dir):
        shutil.rmtree(ocr_dir)
    os.makedirs(ocr_dir)

    # 遍历目标crop目录处理
    recognize_dir = os.path.join(page_dir, "recognize")
    crop_names = os.listdir(recognize_dir)
    
    for crop_name in crop_names:
        crop_dir = os.path.join(recognize_dir, crop_name)
        create_cell_img(crop_dir)
        ocr_cells(crop_dir)
        shutil.move(os.path.join(crop_dir, "ocr"),
                    os.path.join(page_dir, "ocr", crop_name))

def get_text(dir : str):
    page_names = os.listdir(dir)
    for page_name in page_names:
        ic(page_name)
        ocr_page(os.path.join(dir, page_name))

In [5]:
get_text("./results_2")

ic| page_name: '3'


[2024/05/15 23:12:43] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.35640811920166016
[2024/05/15 23:12:43] ppocr DEBUG: cls num  : 1, elapsed : 0.027420997619628906
[2024/05/15 23:12:43] ppocr DEBUG: rec_res num  : 1, elapsed : 0.04473519325256348
[2024/05/15 23:12:43] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.025362491607666016
[2024/05/15 23:12:43] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:12:43] ppocr DEBUG: rec_res num  : 0, elapsed : 9.5367431640625e-07
[2024/05/15 23:12:43] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.016780853271484375
[2024/05/15 23:12:43] ppocr DEBUG: cls num  : 1, elapsed : 0.0030486583709716797
[2024/05/15 23:12:43] ppocr DEBUG: rec_res num  : 1, elapsed : 0.005580902099609375
[2024/05/15 23:12:43] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.008857250213623047
[2024/05/15 23:12:43] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:12:43] ppocr DEBUG: rec_res num  : 0, elapsed : 7.152557373046875e-07
[2024/05/15 23:12:43] ppocr DEBUG: dt_boxes num

ic| page_name: '30'


[2024/05/15 23:12:51] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.008064746856689453
[2024/05/15 23:12:51] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:12:51] ppocr DEBUG: rec_res num  : 0, elapsed : 7.152557373046875e-07
[2024/05/15 23:12:51] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.007448673248291016
[2024/05/15 23:12:51] ppocr DEBUG: cls num  : 1, elapsed : 0.007796049118041992
[2024/05/15 23:12:51] ppocr DEBUG: rec_res num  : 1, elapsed : 0.004964590072631836
[2024/05/15 23:12:51] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.015735149383544922
[2024/05/15 23:12:51] ppocr DEBUG: cls num  : 1, elapsed : 0.0032219886779785156
[2024/05/15 23:12:51] ppocr DEBUG: rec_res num  : 1, elapsed : 0.011820554733276367
[2024/05/15 23:12:51] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.012838363647460938
[2024/05/15 23:12:51] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:12:51] ppocr DEBUG: rec_res num  : 0, elapsed : 4.76837158203125e-07
[2024/05/15 23:12:51] ppocr DEBUG: dt_boxes 

ic| page_name: '4'


...

[2024/05/15 23:13:52] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.012336492538452148
[2024/05/15 23:13:52] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:13:52] ppocr DEBUG: rec_res num  : 0, elapsed : 7.152557373046875e-07
[2024/05/15 23:13:52] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.011222124099731445
[2024/05/15 23:13:52] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:13:52] ppocr DEBUG: rec_res num  : 0, elapsed : 7.152557373046875e-07
[2024/05/15 23:13:52] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.014526128768920898
[2024/05/15 23:13:52] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:13:52] ppocr DEBUG: rec_res num  : 0, elapsed : 7.152557373046875e-07
[2024/05/15 23:13:52] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.006505250930786133
[2024/05/15 23:13:52] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:13:52] ppocr DEBUG: rec_res num  : 0, elapsed : 7.152557373046875e-07
[2024/05/15 23:13:52] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.008522033691406

ic| page_name: '19'


[2024/05/15 23:13:55] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.011614799499511719
[2024/05/15 23:13:55] ppocr DEBUG: cls num  : 1, elapsed : 0.004637956619262695
[2024/05/15 23:13:55] ppocr DEBUG: rec_res num  : 1, elapsed : 0.009775400161743164
[2024/05/15 23:13:55] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.009850025177001953
[2024/05/15 23:13:55] ppocr DEBUG: cls num  : 1, elapsed : 0.0032689571380615234
[2024/05/15 23:13:55] ppocr DEBUG: rec_res num  : 1, elapsed : 0.0046498775482177734
[2024/05/15 23:13:55] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.010262727737426758
[2024/05/15 23:13:55] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:13:55] ppocr DEBUG: rec_res num  : 0, elapsed : 4.76837158203125e-07
[2024/05/15 23:13:55] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.007245779037475586
[2024/05/15 23:13:55] ppocr DEBUG: cls num  : 2, elapsed : 0.006840705871582031
[2024/05/15 23:13:55] ppocr DEBUG: rec_res num  : 2, elapsed : 0.008895397186279297
[2024/05/15 23:13:55] ppo

ic| page_name: '25'


[2024/05/15 23:13:57] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.018156766891479492
[2024/05/15 23:13:57] ppocr DEBUG: cls num  : 1, elapsed : 0.0061664581298828125
[2024/05/15 23:13:57] ppocr DEBUG: cls num  : 1, elapsed : 0.0061664581298828125
[2024/05/15 23:13:57] ppocr DEBUG: rec_res num  : 1, elapsed : 0.005203723907470703
[2024/05/15 23:13:57] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.010703802108764648
[2024/05/15 23:13:57] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:13:57] ppocr DEBUG: rec_res num  : 0, elapsed : 4.76837158203125e-07
[2024/05/15 23:13:57] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.007608652114868164
[2024/05/15 23:13:57] ppocr DEBUG: cls num  : 1, elapsed : 0.0032792091369628906
[2024/05/15 23:13:57] ppocr DEBUG: rec_res num  : 1, elapsed : 0.009897708892822266
[2024/05/15 23:13:57] ppocr DEBUG: dt_boxes num : 0, elapsed : 0.014482736587524414
[2024/05/15 23:13:57] ppocr DEBUG: cls num  : 0, elapsed : 0
[2024/05/15 23:13:57] ppocr DEBUG: rec_res num 